# Remove uncompilable file

## Load solidity file

In [94]:
import os
os.chdir("/home/hieuvd/lvdthieu/CodeGen/")

In [95]:
import pandas as pd
train_file = pd.read_parquet("./data/solfile/train_file.parquet", engine="fastparquet")
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167310 entries, 0 to 167309
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         167310 non-null  object
 1   file_address      167310 non-null  object
 2   source_code       167310 non-null  object
 3   abi               167310 non-null  object
 4   compiler_version  167310 non-null  object
 5   library           167310 non-null  object
dtypes: object(6)
memory usage: 7.7+ MB


In [96]:
test_file = pd.read_parquet("./data/solfile/test_file.parquet", engine="fastparquet")
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18594 entries, 0 to 18593
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         18594 non-null  object
 1   file_address      18594 non-null  object
 2   source_code       18594 non-null  object
 3   abi               18594 non-null  object
 4   compiler_version  18594 non-null  object
 5   library           18594 non-null  object
dtypes: object(6)
memory usage: 871.7+ KB


## Filter out file with "import" word

In [97]:
test_file = test_file[~(test_file["source_code"].str.contains("import", regex=False))]\
    [["file_name", "file_address", "source_code", "compiler_version"]]\
        .reset_index(drop=True)
                        
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13642 entries, 0 to 13641
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         13642 non-null  object
 1   file_address      13642 non-null  object
 2   source_code       13642 non-null  object
 3   compiler_version  13642 non-null  object
dtypes: object(4)
memory usage: 426.4+ KB


In [98]:
train_file = train_file[~(train_file["source_code"].str.contains("import", regex=False))]\
    [["file_name", "file_address", "source_code", "compiler_version"]]\
        .reset_index(drop=True)
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127351 entries, 0 to 127350
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         127351 non-null  object
 1   file_address      127351 non-null  object
 2   source_code       127351 non-null  object
 3   compiler_version  127351 non-null  object
dtypes: object(4)
memory usage: 3.9+ MB


## Standardize compiler_version name

In [99]:
train_file["compiler_version"] = train_file["compiler_version"].apply(lambda x: x.split('+')[0].split('-')[0][1:])
train_file["compiler_version"]

0         0.4.20
1         0.8.11
2         0.6.12
3          0.8.7
4         0.4.24
           ...  
127346    0.6.11
127347    0.5.17
127348    0.4.19
127349    0.6.12
127350    0.5.17
Name: compiler_version, Length: 127351, dtype: object

In [100]:
test_file["compiler_version"] = test_file["compiler_version"].apply(lambda x: x.split('+')[0].split('-')[0][1:])
test_file["compiler_version"]

0        0.4.12
1        0.4.25
2        0.6.12
3        0.6.11
4         0.7.0
          ...  
13637     0.5.9
13638    0.6.12
13639    0.6.12
13640     0.8.1
13641     0.6.6
Name: compiler_version, Length: 13642, dtype: object

## Filter compiler with version < 0.4.11

In [101]:
def filter(stdized_version):
    version = list(map(int, stdized_version.split('.')))
    if version[0] == 0:
        if version[1] == 4:
            if version[2] < 11:
                return False
        if version[2] < 4:
            return False
    return True

In [102]:
test_file = test_file[test_file["compiler_version"].apply(filter)].reset_index(drop=True)
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12078 entries, 0 to 12077
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         12078 non-null  object
 1   file_address      12078 non-null  object
 2   source_code       12078 non-null  object
 3   compiler_version  12078 non-null  object
dtypes: object(4)
memory usage: 377.6+ KB


In [103]:
train_file = train_file[train_file["compiler_version"].apply(filter)].reset_index(drop=True)
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113462 entries, 0 to 113461
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         113462 non-null  object
 1   file_address      113462 non-null  object
 2   source_code       113462 non-null  object
 3   compiler_version  113462 non-null  object
dtypes: object(4)
memory usage: 3.5+ MB


## Test compile

In [104]:
os.chdir("/home/hieuvd/lvdthieu/CodeGen")

In [105]:
compiler_versions = list(set(train_file["compiler_version"].tolist() + test_file["compiler_version"].tolist()))
len(compiler_versions)

58

In [106]:
compiler_config = ""
for version in compiler_versions:
    compiler_config += '{version: "' + version + '"},\n'
with open("compiler_config.txt", "w") as f:
    f.write(compiler_config)

In [107]:
for i in range(100):
    source = test_file.loc[i, "source_code"]
    with open(f"./hardhat/contracts/sample_{i}.sol", "w") as f:
        f.write(source)

In [108]:
os.system("pwd")

/home/hieuvd/lvdthieu/CodeGen


0

In [109]:
os.chdir("hardhat/contracts/")

In [ ]:
# Remove unsatisfied file because of fault in defining pragma
wrong_file_list = [10471, 10552, 11668, 11729, 11884, 1857, 2106, 3165, 3352, 4069, 4259, 4617, 5572, 5739, 
                   619, 6485, 6554, 6566, 6707, 7646, 8374, 8457, 8701, 9136, 9796, 9818]
for file in wrong_file_list:
    os.system(f"rm sample_{file}.sol")

In [124]:
%%bash
rm sample_29.sol

In [125]:
%%time
os.system("npx hardhat compile")

contracts/sample_10.sol: Warning: SPDX license identifier not provided in source file. Before publishing, consider adding a comment containing "SPDX-License-Identifier: <SPDX-License>" to each source file. Use "SPDX-License-Identifier: UNLICENSED" for non-open-source code. Please see https://spdx.org for more information.

contracts/sample_16.sol: Warning: SPDX license identifier not provided in source file. Before publishing, consider adding a comment containing "SPDX-License-Identifier: <SPDX-License>" to each source file. Use "SPDX-License-Identifier: UNLICENSED" for non-open-source code. Please see https://spdx.org for more information.

contracts/sample_25.sol: Warning: SPDX license identifier not provided in source file. Before publishing, consider adding a comment containing "SPDX-License-Identifier: <SPDX-License>" to each source file. Use "SPDX-License-Identifier: UNLICENSED" for non-open-source code. Please see https://spdx.org for more information.

contracts/sample_3.sol: W


CPU times: user 35.9 ms, sys: 308 ms, total: 344 ms
Wall time: 24 s


256

## Store datase

In [129]:
os.chdir("/home/hieuvd/lvdthieu/CodeGen")

In [130]:
test_file.to_parquet("./data/sol-file-v2/test_file.parquet", engine="fastparquet")
train_file.to_parquet("./data/sol-file-v2/train_file.parquet", engine="fastparquet")

In [127]:
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12078 entries, 0 to 12077
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         12078 non-null  object
 1   file_address      12078 non-null  object
 2   source_code       12078 non-null  object
 3   compiler_version  12078 non-null  object
dtypes: object(4)
memory usage: 377.6+ KB


In [132]:
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113462 entries, 0 to 113461
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         113462 non-null  object
 1   file_address      113462 non-null  object
 2   source_code       113462 non-null  object
 3   compiler_version  113462 non-null  object
dtypes: object(4)
memory usage: 3.5+ MB


# Optimize parquet file

In [ ]:
def optimized_parquet(source):
    df = pd.read_parquet(source, engine="fastparquet")
    df.to_parquet(source, engine="fastparquet")